In [1]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [6]:
!which pip

/usr/local/bin/pip


In [3]:
!pip install bayesian_torch

Defaulting to user installation because normal site-packages is not writeable
  Using cached bayesian_torch-0.3.0-py3-none-any.whl (48 kB)
  Using cached torchvision-0.14.1-cp310-cp310-manylinux1_x86_64.whl (24.2 MB)


In [108]:
ls $FOLDER_UQ_PLM/"Code"

__pycache__/     run_juwels.sh*   submit_dev.sh        test.py
info.py          run_juwels.sh~*  submit_dev.sh~       train.py
logs/            run_test.sh*     submit_evaluate.sh   util.py
model.py         run_test.sh~*    submit_evaluate.sh~
model_sto_0.pkl  submit.sh        submit_test.sh
run.sh           submit.sh~       submit_test.sh~


In [113]:
import sys
import pickle
import matplotlib.pyplot as plt
import pickle
import pathlib
import collections
import os
import importlib

def load_module_by_path(file_path, module_name, module_alias):    
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_alias] = module
    spec.loader.exec_module(module)

import numpy as np
import warnings
import dataclasses
import sklearn.metrics
import scipy.stats
import torch.nn.functional
FOLDER_UQ_PLM = pathlib.Path("/p/project/deepacf/maelstrom/ehlert1/UQ-PLM/")
sys.path.append(FOLDER_UQ_PLM / "Code/")
import model, util
load_module_by_path(FOLDER_UQ_PLM / "Code/test.py", "test", "test_uqplm")
import info as info_package

In [116]:
import test_uqplm

In [59]:
os.environ["FOLDER_UQPLM_DATA"]="/p/scratch/deepacf/maelstrom/maelstrom_data/ap2/data/UQ-PLM/"

In [28]:
FOLDER_RESULTS = FOLDER_UQ_PLM / "Result/Task4/electra_base-fl/"

In [29]:
def load_pickle(filename):
    with open(filename, "rb") as input_file:
        return pickle.load(input_file)

In [31]:
model = load_pickle(FOLDER_RESULTS / r"model_det_0.pkl")

In [98]:
os.chdir("/p/project/deepacf/maelstrom/ehlert1/UQ-PLM/notebooks")

In [99]:
@dataclasses.dataclass
class Args:
    task_id: str
    model_name: str
    stage: str

args = Args(task_id="Task4", model_name="electra_base-fl", stage="test")
info = info_package.Info(args)
inputs_list = util.load(args, info, args.stage)
all_labels, all_probs, all_scores = {}, collections.defaultdict(list), collections.defaultdict(list)


In [100]:
info.DIR_OUTPUT

'/p/project/deepacf/maelstrom/ehlert1/UQ-PLM/notebooks/../Result/Task4/electra_base-fl'

In [ ]:
def myprint(to_print, mode):
    print(to_print)

for model_id, model_file in enumerate(info.FILE_MODELS[info.VERSION_DET]):
        myprint(f'Load {info.VERSION_DET} Model {model_id}', info.FILE_STDOUT)
        model = pickle.load(open(model_file, 'rb')).to(info.DEVICE_GPU)
        myprint('-'*20, info.FILE_STDOUT)
        
        in_confs = None; vanilla_logits = {}
        for type, inputs in zip(info.TYPE_TESTS, inputs_list[1:]):
            myprint(f'Uncertainty for {type} Data via {info.VERSION_DET} Model {model_id} and {info.METHOD_VANILLA}', info.FILE_STDOUT)
            each_logits, each_labels = test_uqplm.process(info, info.METHOD_VANILLA, inputs, model, if_eval=True, num_mc=1)
            each_probs = torch.nn.functional.softmax(each_logits, dim=-1); vanilla_logits[type] = each_logits
            each_confs, each_scores = test_uqplm.evaluate(args, info, type, each_probs, each_labels, in_confs=in_confs)
            test_uqplm.feed(info, type, info.METHOD_VANILLA, all_labels, all_probs, all_scores, each_labels, each_probs, each_scores)
            if type == info.TYPE_TEST_IN: in_confs = each_confs
        myprint('-'*20, info.FILE_STDOUT)

Load det Model 0
--------------------
Uncertainty for test_in Data via det Model 0 and Vanilla
Uncertainty for test_out Data via det Model 0 and Vanilla
--------------------
Load det Model 1
--------------------
Uncertainty for test_in Data via det Model 1 and Vanilla
Uncertainty for test_out Data via det Model 1 and Vanilla
--------------------
Load det Model 2
--------------------
Uncertainty for test_in Data via det Model 2 and Vanilla
Uncertainty for test_out Data via det Model 2 and Vanilla
--------------------
Load det Model 3
--------------------
Uncertainty for test_in Data via det Model 3 and Vanilla
Uncertainty for test_out Data via det Model 3 and Vanilla
